In [1]:
import pandas as pd
import torch
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)
from datasets import Dataset
from sklearn.metrics import f1_score, classification_report
import numpy as np
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

/Users/george/Documents/projects/LCT_2025/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data

In [4]:
df = pd.read_pickle("../data/df_final_v2.pkl").drop(columns = ['predicted_classes_new'])

In [5]:
df.head(3)

,index,id,title,text,dateCreate,product,sub_product,source,text_clean,review,predicted_classes_refined,final_classes
0,0,992651,не выполняют условия акции,в апреле 2025 года я рекомендовала дебетовую к...,2025-08-29T23:30:38.746003Z,debitCards,Умная карта (Премиум),sravni,в апреле года я рекомендовала дебетовую карту ...,не выполняют условия акции в апреле года я ре...,"[1, 3, 14, 15]","[debitCards, cashbackPromo, notifications, ref..."
1,1,998360,жалоба на услугу газпром бонус премиум,купил услугу газпром бонус премиум за 2 цель б...,2025-09-15T09:38:13.34818Z,debitCards,Отсутствует,sravni,купил услугу газпром бонус премиум за цель был...,жалоба на услугу газпром бонус премиум купил ...,[],[debitCards]
2,2,993744,банк не отвечает за слова своих сотрудников не...,хочу поделиться историей которая убила моё дов...,2025-09-02T23:21:16.507166Z,debitCards,Умная карта,sravni,хочу поделиться историей которая моё к газпром...,банк не отвечает за слова своих сотрудников не...,"[1, 6, 15]","[debitCards, cashbackPromo, earlyRepayment, re..."


# Model

In [5]:
model_name = "blanchefort/rubert-base-cased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [6]:
device = 0 if torch.cuda.is_available() else -1
print(f"Используем устройство: {'GPU' if device == 0 else 'CPU'}")

Используем устройство: CPU


In [7]:
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer,
    truncation=True,
    max_length=512,
    device=device   # ✨ вот ключевая строка
)

Device set to use cpu


In [15]:
tqdm.pandas()
df["sentiment"] = df["text"].progress_apply(lambda x: sentiment_analyzer(x)[0]["label"])

100%|██████████| 50357/50357 [11:40<00:00, 71.90it/s]


In [17]:
df['sentiment'].value_counts()

sentiment
NEGATIVE    33915
POSITIVE    11667
NEUTRAL      4775
Name: count, dtype: int64

In [39]:
df.to_csv('../data/final_dataset/sentiment_df.csv', index = False)

## Test

In [23]:
data_path = '../data/'

banki = pd.read_csv(data_path + 'banki_reviews_v2.csv').drop(columns='Unnamed: 0')
banki

,id,title,text,grade,agentAnswerText,dateCreate,product
0,12567677,Развод со ставкой,<p>Ради безопасности решил часть средств вложи...,2.0,NaN,2025-09-22 20:37:22,debitcards
1,12567381,"Банкомат не выдал наличные, а деньги списал со...","<p>Сегодня, 22 сентяюря 2025 года в 12-55 (Сар...",2.0,NaN,2025-09-22 17:37:05,debitcards
2,12567376,Оформление карты по реферальной ссылке,<p>Оформил заказ на дебетовую карту дочери по ...,3.0,NaN,2025-09-22 17:34:14,debitcards
3,12567253,Получение дебетовой карты ульяновск,"<p>Хочу поблагодарить представителя Ксению, по...",5.0,Здравствуйте!\n\nБлагодарим за положительный о...,2025-09-22 16:37:28,debitcards
4,12567156,Получение дебетовой карты,<p>Здравствуйте. <br>Заказала карту Газпромбан...,5.0,Здравствуйте!\n\nБлагодарим за высокую оценку....,2025-09-22 15:54:51,debitcards
...,...,...,...,...,...,...,...
45831,11323939,Обман и навязывание услуг,Добрый вечер Уважаемое руководство банка!\r\nС...,1.0,Здравствуйте!\r\n\r\nВаше обращение принято в ...,2024-01-10 22:17:28,businessdeposits
45832,11321849,Дикий запад,"<p>Карточный счёт по не действующей, неактивно...",1.0,Здравствуйте!\r\n\r\nВаше обращение принято в ...,2024-01-09 17:14:53,businessdeposits
45833,11282806,Депозит,Я являюсь клиентом Газпромбанк несколько лет. ...,5.0,"Здравствуйте!\r\n\r\nМы рады, что вы выбрали н...",2023-12-15 22:14:57,businessdeposits
45834,11266609,Наплевательское отношение к клиентам,Оформила онлайн-заявку на получение дебетовой ...,1.0,"Здравствуйте!\r\n\r\nНам жаль, что сложилась т...",2023-12-10 10:05:20,businessdeposits


In [24]:
banki_sent = banki.merge(df[['id', 'sentiment']], on = 'id', how = 'left')

In [28]:
banki_sent = banki_sent.dropna(subset=["sentiment"]).reset_index(drop=True)

In [ ]:
banki_sent

,id,title,text,grade,agentAnswerText,dateCreate,product,sentiment
0,12567677,Развод со ставкой,<p>Ради безопасности решил часть средств вложи...,2.0,NaN,2025-09-22 20:37:22,debitcards,NEGATIVE
1,12567677,Развод со ставкой,<p>Ради безопасности решил часть средств вложи...,2.0,NaN,2025-09-22 20:37:22,debitcards,NEGATIVE
2,12567381,"Банкомат не выдал наличные, а деньги списал со...","<p>Сегодня, 22 сентяюря 2025 года в 12-55 (Сар...",2.0,NaN,2025-09-22 17:37:05,debitcards,NEGATIVE
3,12567376,Оформление карты по реферальной ссылке,<p>Оформил заказ на дебетовую карту дочери по ...,3.0,NaN,2025-09-22 17:34:14,debitcards,NEGATIVE
4,12567253,Получение дебетовой карты ульяновск,"<p>Хочу поблагодарить представителя Ксению, по...",5.0,Здравствуйте!\n\nБлагодарим за положительный о...,2025-09-22 16:37:28,debitcards,POSITIVE
...,...,...,...,...,...,...,...,...
54533,11626839,Долгое разбирательство,18.06.2024 открыл накопительный счёт по ставке...,1.0,"Здравствуйте!\r\n\r\nКак обсудили по телефону,...",2024-07-10 22:14:56,businessdeposits,NEGATIVE
54534,11323939,Обман и навязывание услуг,Добрый вечер Уважаемое руководство банка!\r\nС...,1.0,Здравствуйте!\r\n\r\nВаше обращение принято в ...,2024-01-10 22:17:28,businessdeposits,NEGATIVE
54535,11282806,Депозит,Я являюсь клиентом Газпромбанк несколько лет. ...,5.0,"Здравствуйте!\r\n\r\nМы рады, что вы выбрали н...",2023-12-15 22:14:57,businessdeposits,POSITIVE
54536,11266609,Наплевательское отношение к клиентам,Оформила онлайн-заявку на получение дебетовой ...,1.0,"Здравствуйте!\r\n\r\nНам жаль, что сложилась т...",2023-12-10 10:05:20,businessdeposits,NEGATIVE


In [32]:
banki_sent = banki_sent[~banki_sent['grade'].isin([1.466, 4.759])]

In [33]:
banki_sent['grade'].value_counts()

grade
1.0    28530
5.0    13706
2.0     4495
3.0     1510
4.0     1079
Name: count, dtype: int64

In [34]:
pivot = banki_sent.pivot_table(
    index="sentiment",
    columns="grade",
    values="id",
    aggfunc="count",
    fill_value=0
)

print(pivot)

grade        1.0   2.0   3.0  4.0    5.0
sentiment                               
NEGATIVE   25292  3778  1097  396    960
NEUTRAL     2616   543   282  212    826
POSITIVE     622   174   131  471  11920


# Inference

In [ ]:
def get_sentiment(text):
    """
    Возвращает тональность текста: 'негативный', 'нейтральный', 'позитивный'.
    
    text : str : произвольный текст на русском
    """
    if not text or not text.strip():
        return None  # пустой текст

    result = sentiment_analyzer(text)[0]["label"]

    # Приводим к русским меткам
    label_map = {
        "NEGATIVE": "негативный",
        "NEUTRAL": "нейтральный",
        "POSITIVE": "позитивный"
    }

    return label_map.get(result, None)

In [8]:
text1 = "Банк отличный, очень доволен сервисом!"
text2 = "Очень плохое обслуживание, не рекомендую."
text3 = "Ну, в целом нормально, ничего особенного."

print(get_sentiment(text1))  # ➜ 'позитивный'
print(get_sentiment(text2))  # ➜ 'негативный'
print(get_sentiment(text3))  # ➜ 'нейтральный'


позитивный
негативный
нейтральный


In [38]:
df

,index,id,title,text,dateCreate,product,sub_product,source,text_clean,review,predicted_classes_refined,final_classes,num_tokens,sentiment
0,0,992651,не выполняют условия акции,в апреле 2025 года я рекомендовала дебетовую к...,2025-08-29T23:30:38.746003Z,debitCards,Умная карта (Премиум),sravni,в апреле года я рекомендовала дебетовую карту ...,не выполняют условия акции в апреле года я ре...,"[1, 3, 14, 15]","[debitCards, cashbackPromo, notifications, ref...",262,NEGATIVE
1,1,998360,жалоба на услугу газпром бонус премиум,купил услугу газпром бонус премиум за 2 цель б...,2025-09-15T09:38:13.34818Z,debitCards,Отсутствует,sravni,купил услугу газпром бонус премиум за цель был...,жалоба на услугу газпром бонус премиум купил ...,[],[debitCards],155,NEGATIVE
2,2,993744,банк не отвечает за слова своих сотрудников не...,хочу поделиться историей которая убила моё дов...,2025-09-02T23:21:16.507166Z,debitCards,Умная карта,sravni,хочу поделиться историей которая моё к газпром...,банк не отвечает за слова своих сотрудников не...,"[1, 6, 15]","[debitCards, cashbackPromo, earlyRepayment, re...",309,NEGATIVE
3,3,999494,не выполняют условия акции,в июне 2025 года я порекомендовал премиальную ...,2025-09-18T08:10:19.954986Z,debitCards,Отсутствует,sravni,в июне года я порекомендовал премиальную дебет...,не выполняют условия акции в июне года я поре...,"[1, 3, 14]","[debitCards, cashbackPromo, notifications, ref...",206,NEGATIVE
4,4,992291,странный банк,добрый день в связи с устройством на новую раб...,2025-08-28T19:29:51.696341Z,debitCards,Отсутствует,sravni,день в связи с устройством на новую работу мне...,странный банк день в связи с устройством на но...,"[9, 15]","[debitCards, depositAccess, remoteService]",230,NEGATIVE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50352,50352,10885124,под видом кредита 12 9 пытаются впарить кредит...,газпромбанк тут заманивал кредитом одобренным ...,2023-04-15 23:33:45,individual,Отсутствует,banki,газпромбанк тут заманивал кредитом под пришел ...,под видом кредита 12 9 пытаются впарить кредит...,"[6, 13]","[other, earlyRepayment, loanIssues]",58,NEGATIVE
50353,50353,10872392,неполное изменение номера в офисе отделении,ситуация довольно странная для входа в интерне...,2023-03-30 12:39:01,individual,Отсутствует,banki,ситуация довольно странная для входа в интерне...,неполное изменение номера в офисе отделении си...,[],[other],142,NEGATIVE
50354,50354,10852603,один из отличных банков,в банке нравится все и обслуживание на отлично...,2023-03-10 15:04:12,individual,Отсутствует,banki,в банке все и обслуживание на отлично и дистан...,один из отличных банков в банке все и обслужив...,[],[other],56,POSITIVE
50355,50355,10852597,обращение в контакт центр 10 баллов,обращение в контакт центр 10 баллов а вот рабо...,2023-03-10 14:57:02,individual,Отсутствует,banki,обращение в контакт центр баллов а вот работа ...,обращение в контакт центр 10 баллов обращение ...,[],[other],62,POSITIVE
